# Investigating the Optimality Gap for Gitcoin Grants

On this notebook, we implement a workflow for studying the optimality gap distribution on Gitcoin Grants Round 8 data.

The optimality gap is a measure of how a particular community is optimized in regards to the edge arragements towards a given utility function.

$OptimalityGap = 1 - \frac{m_r}{m_o}$

As a rule of thumb, we can expect the following additional match if a community adopt a optimal structure:

Optimality Gap | Additional match
- | -
0.9 | 10x
0.75 | 4x
0.5 | 2x
0.25 | 1.33x
0.1 | 1.11x

Source: https://hackmd.io/QCCJWZE0Ru27X_GRk6UKjQ

## Dependences

In [1]:
%load_ext autotime
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

time: 75 ms (started: 2021-03-12 12:20:38 -03:00)


In [ ]:
# Library dependences
import cloudpickle
import holoviews as hv
import plotly.express as px
import networkx as nx
import pandas as pd
import numpy as np

# Local dependences
from model_gitcoin.parts.utils import plot_contributions
from qf_research.functions import total_amount
from qf_research.functions import contributions_to_graph
from qf_research.definitions import optimality_gap_per_grant
from qf_research.definitions import amount_per_grant

In [ ]:
hv.extension('bokeh')

## Load data from Pickle file

First we need to load the pickled simulation result from the cadCAD simulation. If you don't have that file, then you are going to need to run the model_gitcoin. Refer to the `README.md` doc for instructions for doing that.

In [ ]:
from env_config import PICKLE_PATH
import os

PICKLE_PATH = os.path.join("..", PICKLE_PATH)

# Load the data generated by the cadCAD simulation
with open(PICKLE_PATH, 'rb') as fid:
    result = cloudpickle.load(fid)

In [ ]:
# Get the contributions on the third timestep
result.iloc[2].contributions

## Overview of the graph

In [ ]:
result.head(3)

plot_contributions(result.contributions.iloc[GRAPH_INDEX])

fig = (hv.Graph.from_networkx(G, nx.layout.fruchterman_reingold_layout, k=1)
         .opts(tools=['hover']))

colors = ['#000000']+hv.Cycle('Category20').values

fig.opts(cmap=colors, node_color='type')

## Optimality Gap Analysis

### Distribution for a static point on time

In [ ]:
n_iter = 50
GRAPH_INDEX = 200

G = contributions_to_graph(result.iloc[GRAPH_INDEX].contributions)
distribution = optimality_gap_per_grant(G, n_iter=n_iter)

s = pd.Series(distribution)
s.name = 'optimality_gap'
px.histogram(s,
             title=f'Optimality Gap Histogram (t={GRAPH_INDEX}, n_iter={n_iter})',
             nbins=100)

In [ ]:
n_iter = 200
GRAPH_INDEX = 200

G = contributions_to_graph(result.iloc[GRAPH_INDEX].contributions)
distribution = optimality_gap_per_grant(G, n_iter=n_iter)

s = pd.Series(distribution)
s.name = 'optimality_gap'
px.histogram(s,
             title=f'Optimality Gap Histogram (t={GRAPH_INDEX}, n_iter={n_iter})',
             nbins=100)

#### Sensitivity of Optimality Gap towards the number of optimization steps

In [ ]:
N_ITER = [10, 100, 1000]
GRAPH_INDEX = 200

output = []
for n_iter in N_ITER:
    G = contributions_to_graph(result.iloc[GRAPH_INDEX].contributions)
    distribution = optimality_gap_per_grant(G, n_iter=n_iter)
    s = pd.Series(distribution)
    s.name = 'optimality_gap'
    it_df = s.reset_index().assign(n_iter=n_iter)
    output.append(it_df)

# Plot a 2D histogram comparing the optimality gap distribution vs number of iterations
fig_df = pd.concat(output)
fig = px.density_heatmap(fig_df,
                         x=fig_df.n_iter.astype(str),
                         y='optimality_gap',
                         labels={'x': 'Optimization iterations'},
                         title=f'Optimality Gap distribution across iterations (t={GRAPH_INDEX})',
                         nbinsy=50)

fig.show()

### Dynamical Network Analysis

In [ ]:
graphs = result.contributions.iloc[1:1000:250].map(contributions_to_graph)
optimality_gap_per_time = graphs.map(lambda x: optimality_gap_per_grant(x, n_iter=5))
amount_per_time = graphs.map(amount_per_grant)
print('done')

#### Heatmap of the Optimality Gap over Time

In [ ]:

N_bins = 30
target = optimality_gap_per_time

# Get the maximum value inside the distributions df
max_x: float = max(max(x if not np.isnan(x) else 0 
                       for x 
                       in x.values()) 
                   for x 
                   in target)

N = len(target)

# Make a manual histogram
values = np.array([np.histogram(list(d.values()), bins=N_bins, range=(0, max_x))[0]
                   for d in target]).T

# Calculate ranges
y_range = (np.arange(N_bins) / N_bins) * max_x
x_range = np.arange(N) / N

# Plot matrix plot
px.imshow(values, 
          y=y_range, 
          x=x_range, 
          title='Optimality Gap Over Time',
          labels={'x': 'Time',
                  'y': 'Optimality Gap'})

#### Scatter plot of the Optimality Gap versus Grant Match over Time

In [ ]:
def series_of_dict_to_df(s, index_name, series_name):

    def f(df):
        index_col = df.columns[-2]
        series_col = df.columns[-1]
        col_map = {index_col: index_name,
                   series_col: series_name}
        return df.rename(columns=col_map)

    return (s.apply(pd.Series)
             .stack()
             .reset_index()
             .pipe(f))

df_1 = series_of_dict_to_df(optimality_gap_per_time, 'grant', 'optimality_gap')
df_2 = series_of_dict_to_df(amount_per_time, 'grant', 'amount')
index_cols = list(df_1.columns[:-1])
df_3 = df_1.set_index(index_cols).join(df_2.set_index(index_cols))

In [ ]:
fig = px.line(df_3.reset_index(),
                 x='optimality_gap',
                 y='amount',
                 color='grant',
                 log_y=True)
fig.update_layout(showlegend=False)
fig.show()

## Conclusions